In [ ]:
from pathlib import Path
import json
import pandas as pd

# Top transaction Data District

In [ ]:
def Top_Transcations_Distrit():

    root_dir = Path("../data/top/transaction/country/india/state/")

    lis = []
    for i in root_dir.rglob("*.json"):
        # print(i)
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)
            
            state = i.parent.parent.name
            year = i.parent.name
            
            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({"state": state, "year": year, "quarter": quarter, "data": dataset})



    lis2 = []
    for j in lis:
        for k in j['data']['data']['districts']:
            amount_value = k['metric']['amount']
            amount_formatted = f'{amount_value: .2f}'
            data = dict(state = j['state'],
                        year = j['year'],
                        quarter = j['quarter'],
                        disrict_name = k['entityName'],
                        type = k['metric']['type'],
                        count = k['metric']['count'],
                        amount = amount_formatted,
                        timestamp = j['data']['responseTimestamp'])
            lis2.append(data)
    
    return lis2





In [ ]:
TopDistrictName =pd.DataFrame(Top_Transcations_Distrit())
TopDistrictName['year'] = TopDistrictName['year'].astype(int)
TopDistrictName['amount'] = TopDistrictName['amount'].astype(float)
TopDistrictName['timestamp'] = pd.to_datetime(TopDistrictName['timestamp'], unit='ms')
TopDistrictName['timestamp'] = TopDistrictName['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
TopDistrictName.info()

In [30]:
import mysql.connector
from tabulate import tabulate
import numpy as np

In [ ]:
db = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'balaji',
    database = 'pulse'
)

mycursor = db.cursor(buffered=True)
mycursor

In [ ]:
mycursor.execute("""create table if not exists topdatatransdistrict (
                topDataTransDistrictId int auto_increment primary key, state varchar(255), 
                year int(10), quarter varchar(255), district_name varchar(255), type varchar(10),
                count int(255), amount float, timestamp datetime)""")

In [ ]:
sql = ("""insert into topdatatransdistrict (topDataTransDistrictId, state,year, quarter, 
       district_name, type, count, amount, timestamp) values(%s, %s, %s, %s, %s, %s, %s, %s, %s)
       on duplicate key update
       state = values(state), year = values(year), quarter = values(quarter), 
       district_name = values(district_name), type = values(type), count = values(count), 
       amount = values(amount), timestamp = values(timestamp)""")

for i in TopDistrictName.to_records().tolist():
    mycursor.execute(sql, i)

In [ ]:
db.commit()

# Top transaction Data Pincode

In [14]:
def Top_Transcations_Pincode():

    root_dir = Path("../data/top/transaction/country/india/state/")

    lis = []
    for i in root_dir.rglob("*.json"):
        # print(i)
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)
            
            state = i.parent.parent.name
            year = i.parent.name
            
            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({"state": state, "year": year, "quarter": quarter, "data": dataset})

    lis3 = []
    for l in lis:
        for m in l['data']['data']['pincodes']:
            amount_value = m['metric']['amount']
            amount_formatted = f'{amount_value : .2f}'

            data1 = dict(state = l['state'],
                        year = l['year'],
                        quarter = l['quarter'],
                        pincodes = m['entityName'],
                        type = m['metric']['type'],
                        count = m['metric']['count'],
                        amount = amount_formatted,
                        timestamp = l['data']['responseTimestamp'])
            lis3.append(data1)

    return lis3

In [32]:
TopPincodeName = pd.DataFrame(Top_Transcations_Pincode())
TopPincodeName['year'] = TopPincodeName['year'].astype(int)
# replace nan with 0 and inf with a suitable value(eg -1)
TopPincodeName['pincodes'] = TopPincodeName['pincodes'].fillna(0).replace([np.inf, -np.inf], -1).astype(float).astype(int)
TopPincodeName['amount'] = TopPincodeName['amount'].astype(float)
TopPincodeName['timestamp'] = pd.to_datetime(TopPincodeName['timestamp'], unit='ms')
TopPincodeName['timestamp'] = TopPincodeName['timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")
TopPincodeName

,state,year,quarter,pincodes,type,count,amount,timestamp
0,andaman-&-nicobar-islands,2018,Q1,744101,TOTAL,1622,2.769298e+06,2021-09-01 13:04:46
1,andaman-&-nicobar-islands,2018,Q1,744103,TOTAL,1223,2.238042e+06,2021-09-01 13:04:46
2,andaman-&-nicobar-islands,2018,Q1,744102,TOTAL,969,3.519060e+06,2021-09-01 13:04:46
3,andaman-&-nicobar-islands,2018,Q1,744105,TOTAL,685,1.298561e+06,2021-09-01 13:04:46
4,andaman-&-nicobar-islands,2018,Q1,744104,TOTAL,340,1.039715e+06,2021-09-01 13:04:46
...,...,...,...,...,...,...,...,...
7848,west-bengal,2023,Q2,700001,TOTAL,2918444,6.259877e+09,2023-08-21 12:00:57
7849,west-bengal,2023,Q2,721301,TOTAL,2910870,4.458107e+09,2023-08-21 12:00:57
7850,west-bengal,2023,Q2,700039,TOTAL,2886562,4.662887e+09,2023-08-21 12:00:57
7851,west-bengal,2023,Q2,700015,TOTAL,2774383,4.372975e+09,2023-08-21 12:00:57


In [33]:
mycursor.execute("""create table if not exists topdatatranspincode (
                 topDataTransPincodeId int auto_increment primary key, state varchar(255), 
                 year int(10), quarter varchar(10), pincodes int(10), type varchar(10), 
                 count bigint, amount float, timestamp datetime
                 )""")

In [36]:
sql = ("""insert into topdatatranspincode (topDataTransPincodeId, state, year, quarter, pincodes, type,
       count, amount, timestamp) values(%s, %s, %s, %s, %s, %s, %s, %s, %s)
       on duplicate key update 
       state = values(state), year = values(year), quarter = values(quarter), 
       pincodes = values(pincodes), type = values(type), count = values(count), amount = values(amount),
       timestamp = values(timestamp)""")

for i in TopPincodeName.to_records().tolist():
    mycursor.execute(sql, i)

In [37]:
db.commit()

# Top Users Data District

In [40]:
def Top_Users_District():

    root_dir = Path("../data/top/user/country/india/state")

    lis = []
    for i in root_dir.rglob("*.json"):
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)

            state = i.parent.parent.name
            year = i.parent.name

            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({"state":state, "year":year, "quarter":quarter, 'data': dataset})

    # print(lis)

    lis2 = []
    for j in lis:
        for k in j['data']['data']['districts']:
            data = dict(state = j['state'],
                        year = j['year'],
                        quarter = j['quarter'],
                        district_name = k['name'],
                        registeredUsers = k['registeredUsers'],
                        timestamp = j['data']['responseTimestamp'])
            lis2.append(data)

    return lis2

In [43]:
TopUsersDistrict = pd.DataFrame(Top_Users_District())
TopUsersDistrict['year'] = TopUsersDistrict['year'].astype(int)
TopUsersDistrict['timestamp'] = pd.to_datetime(TopUsersDistrict['timestamp'], unit='ms')
TopUsersDistrict['timestamp'] = TopUsersDistrict['timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")
TopUsersDistrict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6512 entries, 0 to 6511
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   state            6512 non-null   object
 1   year             6512 non-null   int32 
 2   quarter          6512 non-null   object
 3   district_name    6512 non-null   object
 4   registeredUsers  6512 non-null   int64 
 5   timestamp        6512 non-null   object
dtypes: int32(1), int64(1), object(4)
memory usage: 279.9+ KB


In [44]:
mycursor.execute("""create table if not exists topdatausersdistrict (
                 topDataUsersDistrictId int auto_increment primary key, state varchar(255), 
                 year int(10), quarter varchar(10), district_name varchar(255), 
                 registeredUsers bigint, timestamp datetime )""")

In [47]:
sql = ("""insert into topdatausersdistrict (topDataUsersDistrictId, state, year, quarter, 
       district_name, registeredUsers, timestamp) values(%s, %s, %s, %s, %s, %s, %s)
       on duplicate key update 
       state = values(state), year = values(year), quarter = values(quarter), 
       district_name = values(district_name), registeredUsers = values(registeredUsers),
       timestamp = values(timestamp)
       """)

for i in TopUsersDistrict.to_records().tolist():
    mycursor.execute(sql,i)

In [48]:
db.commit()

# Top Users Data Pincode

In [54]:
def Top_Users_Pincode():

    root_dir = Path("../data/top/user/country/india/state")

    lis = []
    for i in root_dir.rglob("*.json"):
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)

            state = i.parent.parent.name
            year = i.parent.name

            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({"state":state, "year":year, "quarter":quarter, 'data': dataset})

    # print(lis)

    lis3 = []
    for l in lis:
        for m in l['data']['data']['pincodes']:
            data = dict(state = l['state'],
                        year = l['year'],
                        quarter = l['quarter'],
                        pincodes = m['name'],
                        registeredUsers = m['registeredUsers'],
                        timestamp = l['data']['responseTimestamp'])
            lis3.append(data)

    return lis3

In [60]:
TopUsersPincode = pd.DataFrame(Top_Users_Pincode())
TopUsersPincode['year'] = TopUsersPincode['year'].astype(int)
TopUsersPincode['pincodes'] = TopUsersPincode['pincodes'].astype(int)
TopUsersPincode['timestamp'] = pd.to_datetime(TopUsersPincode['timestamp'], unit='ms')
TopUsersPincode['timestamp'] = TopUsersPincode['timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")
TopUsersPincode

,state,year,quarter,pincodes,registeredUsers,timestamp
0,andaman-&-nicobar-islands,2018,Q1,744103,1608,2021-09-01 13:04:54
1,andaman-&-nicobar-islands,2018,Q1,744101,1108,2021-09-01 13:04:54
2,andaman-&-nicobar-islands,2018,Q1,744105,1075,2021-09-01 13:04:54
3,andaman-&-nicobar-islands,2018,Q1,744102,1006,2021-09-01 13:04:54
4,andaman-&-nicobar-islands,2018,Q1,744104,272,2021-09-01 13:04:54
...,...,...,...,...,...,...
7849,west-bengal,2023,Q2,700015,117690,2023-08-21 12:02:01
7850,west-bengal,2023,Q2,742304,114746,2023-08-21 12:02:01
7851,west-bengal,2023,Q2,721101,114293,2023-08-21 12:02:01
7852,west-bengal,2023,Q2,700150,110694,2023-08-21 12:02:01


In [62]:
mycursor.execute("drop table topdatauserspincode")

In [63]:
mycursor.execute("""create table if not exists topdatauserspincode (
                 topDataUsersPincodeid int auto_increment primary key, state varchar(255),
                 year int(10), quarter varchar(10), pincodes int, registeredUsers bigint, 
                 timestamp datetime)""")

In [64]:
sql = ("""insert into topdatauserspincode (topDataUsersPincodeId, state, year, quarter, pincodes,
       registeredUsers, timestamp)
       values(%s, %s, %s, %s, %s, %s, %s) 
       on duplicate key update
       state = values(state), year = values(year), quarter = values(quarter), 
       pincodes = values(pincodes), registeredUsers = values(registeredUsers), 
       timestamp = values(timestamp)""")

for i in TopUsersPincode.to_records().tolist():
    mycursor.execute(sql,i)

In [65]:
db.commit()